# Baseline: Bi-LSTM
Author: Markie Wagner

In [9]:
import csv
import pandas as pd
from data.SpotifyDataset import SpotifyDataset

In [16]:
train_file = 'data/train_data.csv'
track_file = 'data/track_feats.csv'
train = pd.read_csv('data/train_data.csv')
track_feats = pd.read_csv('data/track_feats.csv')

In [11]:
train

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,hist_user_behavior_reason_start_trackdone,hist_user_behavior_reason_start_trackerror,hist_user_behavior_reason_end_appload,hist_user_behavior_reason_end_backbtn,hist_user_behavior_reason_end_clickrow,hist_user_behavior_reason_end_endplay,hist_user_behavior_reason_end_fwdbtn,hist_user_behavior_reason_end_logout,hist_user_behavior_reason_end_remote,hist_user_behavior_reason_end_trackdone
0,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,0.000000,1.0,t_1a73002a-d065-4a33-b98a-a3b0443310be,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,0.052632,1.0,t_1a73002a-d065-4a33-b98a-a3b0443310be,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,0.105263,1.0,t_16bc2816-8daf-449a-a0f7-93ee884ffdda,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,0.157895,1.0,t_71d34c2f-1c87-43df-83ba-00832e12194c,0,1,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,0.210526,1.0,t_a66ea088-b357-449a-8a1e-64dd0b8d6cb5,0,1,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323207,63_ffffbd30-c598-46c4-af6a-569403e4bd44,0.789474,1.0,t_6335a948-7d30-4bbd-91e9-76f2592e3a25,1,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2323208,63_ffffbd30-c598-46c4-af6a-569403e4bd44,0.842105,1.0,t_3e2928a7-dcc4-49bf-a21a-9d28cf1bdb6c,1,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2323209,63_ffffbd30-c598-46c4-af6a-569403e4bd44,0.894737,1.0,t_a9aab736-565f-4248-b9b8-9385a90fb6aa,1,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2323210,63_ffffbd30-c598-46c4-af6a-569403e4bd44,0.947368,1.0,t_0df9627d-4ac7-45ff-adc2-dd736e83e099,1,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [17]:
# Pre-process
SpotifyDataset = SpotifyDataset(train_file, track_file)

In [52]:
# Create Bi-LSTM Encoder Class
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

class Encoder(nn.Module):
    def __init__(self, input_dim=67, hid_dim=256, n_layers=1, dropout=None):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_size=input_dim,
                             hidden_size=hid_dim,
                             num_layers=n_layers, 
                             dropout=dropout,
                             bidirectional=True)
        
        #self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size, input_dim]
        
        outputs, (hidden, cell) = self.lstm(src)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell
    

class Decoder(nn.Module):
    def __init__(self, input_dim=29, out_output_dim=1, hid_dim=256, n_layers=1, dropout=None):
        super().__init__()
        
        input_dim = input_dim
        self.output_dim = out_output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hid_dim, n_layers, dropout=dropout)
        
        self.fc_out = nn.Linear(hid_dim, out_output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size, feat dim]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size, feature dim]
        
        ##embedded = self.dropout(self.embedding(input))
        ##embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell
    

In [57]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [64]:
ENC_INPUT_DIM = 67
OUTPUT_DIM = 1
DEC_INPUT_DIM = 29
HID_DIM = 256
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(END_INPUT_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(DEC_INPUT_DIM, OUTPUT_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(67, 256, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (lstm): LSTM(29, 256, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=1, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [65]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [66]:
# Use BiLSTM To Encode
print(SpotifyDataset.__getitem__(3)[0].columns)

Index(['session_position', 'session_length', 'skip_1', 'skip_2', 'skip_3',
       'not_skipped', 'context_switch', 'no_pause_before_play',
       'short_pause_before_play', 'long_pause_before_play',
       'hist_user_behavior_n_seekfwd', 'hist_user_behavior_n_seekback',
       'hist_user_behavior_is_shuffle', 'hour_of_day', 'premium',
       'context_type_catalog', 'context_type_charts',
       'context_type_editorial_playlist', 'context_type_personalized_playlist',
       'context_type_radio', 'context_type_user_collection',
       'hist_user_behavior_reason_start_appload',
       'hist_user_behavior_reason_start_backbtn',
       'hist_user_behavior_reason_start_clickrow',
       'hist_user_behavior_reason_start_endplay',
       'hist_user_behavior_reason_start_fwdbtn',
       'hist_user_behavior_reason_start_playbtn',
       'hist_user_behavior_reason_start_remote',
       'hist_user_behavior_reason_start_trackdone',
       'hist_user_behavior_reason_start_trackerror',
       'hist_u